# Prompt engineering

In [2]:
# import functions from a file stored in a different directory
import sys
sys.path.append('/home/ubuntu/work/therapeutic_accelerator/scripts/utils')
sys.path.append('/home/ubuntu/work/therapeutic_accelerator/scripts/database')

from utils import import_config
from db_tools import db_connection

config, keys = import_config()

engine = db_connection(password = keys["postgres"], host = config["database"]["host"])

# Testing questions / messages

In [18]:
import pandas as pd
prompts = pd.read_csv(
    '/home/ubuntu/work/therapeutic_accelerator/model_work/prompt_engineering/prompts.csv').loc[:, 'Prompt']

# Setup 
Prompt -> split text -> tokenize -> model

In [19]:
from transformers import T5Tokenizer
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

# Text splitter for processing input into chunks
recursive_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ".", "?", "!"],
    chunk_size=500,
    chunk_overlap=20,
    length_function=len,
)

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=500,
    chunk_overlap=20,
    length_function=len
)


max_sequence_length = 1200
embedding_size = 200

T5tokens = T5Tokenizer.from_pretrained(
    't5-base', model_max_length=max_sequence_length)


def split_tokenize(prompt):
    """ Split text into chunks and tokenize each chunk"""
    return [T5tokens.encode(chunk) for chunk in text_splitter.split_text(prompt)]

In [21]:
p = prompts[0]
p_split = split_tokenize(p)
p_split

[[363, 19, 8, 167, 750, 585, 30, 2131, 5045, 6049, 1874, 58, 1]]

Prompt Templates

In [24]:
from getpass import getpass
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

os.environ["OPENAI_API_KEY"] = keys['openai']

# for looking at different indications
llm = OpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["indication"],
    template="What work is being done on {indication}?",
)

NameError: name 'os' is not defined

In [ ]:
# Chain to take user input, format the prompt with it, and then send it to the LLM.
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("pancreatic cancer"))

# User Input

In [ ]:
# get stdin from user
def user_input():
    return input("Send a message: ")

message = user_input()

In [ ]:
message = user_input()

In [ ]:
# Embed Message

In [ ]:
# Create chain of user inputs for context using langchain